In [1]:
%cd /home/pac/gits/phd/mri-online/

/home/pac/gits/phd/mri-online


In [2]:
import yaml
import pprint
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import tikzplotlib as tikz
from results.base import Experience, loader
from results.plot import plot_metrics, make_gif

from project_utils import implot, imsave, load_data
from online.metrics import psnr_ssos, ssim_ssos, ssos

/home/pac/gits/phd/pysap-mri/mri/operators/fourier/non_cartesian.py:27: UserWarning: pynfft python package has not been found. If needed use the master release.
  warnings.warn("pynfft python package has not been found. If needed use "
/home/pac/gits/phd/pysap-mri/mri/operators/fourier/non_cartesian.py:42: UserWarning: gpuNUFFT python package has not been found. If needed please check on how to install in README
  warnings.warn("gpuNUFFT python package has not been found. If needed "


In [3]:
%matplotlib widget
plt.rcParams['figure.figsize'] = (12, 6)
DATA_DIR = "data/"
PLOT_DIR = "plots/"
full_k, real_img, mask_loc, final_mask = load_data(DATA_DIR,1)
square_mask= np.zeros(real_img.shape)
real_img_size = real_img.shape
img_size = [min(real_img.shape)]*2
square_mask[real_img_size[0] // 2 - img_size[0] // 2:real_img_size[0] // 2 + img_size[0] // 2,
            real_img_size[1] // 2 - img_size[1] // 2:real_img_size[1] // 2 + img_size[1] // 2] = 1

In [9]:
with open('results/tested_config.yml') as f:
    tested_cfg = yaml.load(f, Loader=loader)
Experience.save_folder = 'results/simuls/'
Experience.objects.flush()
for cfg in tested_cfg:
    Experience.objects.add(Experience(**cfg))

## Regularisation 
### Forward-Backward

In [10]:
off = Experience.objects.get(mode='and',
                       problem__fourier_type=0,
                       problem__regularisation__class='LASSO',
                       problem__regularisation__weights=1e-8,
                       solver__algo__in=['vanilla'],
                       solver__reg_factor=1)
img = off.xf[-1] 
implot(img,  mask=square_mask, title=f'psnr={psnr_ssos(img,real_img, mask=square_mask):.4f}dB ssim={ssim_ssos(img,real_img, mask=square_mask):.2f}');
off = Experience.objects.get(mode='and',
                       problem__fourier_type=0,
                       problem__regularisation__class='LASSO',
                       problem__regularisation__weights=1e-8,
                       solver__algo__in=['pogm'])
img = off.xf[-1] 
implot(img,  mask=square_mask, title=f'psnr={psnr_ssos(img,real_img, mask=square_mask):.4f}dB ssim={ssim_ssos(img,real_img, mask=square_mask):.2f}');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'pop from an empty set'

In [14]:
offline = Experience.objects.filter(mode='and',
                               problem__fourier_type__in=[0, 1],
                               problem__regularisation__class='LASSO',
                                problem__regularisation__weights__le=1e-8,
                               solver__algo__in=['vanilla'],
                               solver__reg_factor=1,
                               )
offline = offline.union(Experience.objects.filter(mode='and',
                               problem__fourier_type__in=[0, 1],
                               problem__regularisation__class='IdentityProx',
                               solver__algo__in=['vanilla'],
                               solver__reg_factor=1,
                               ))

online2vanilla = Experience.objects.filter(mode='and',
                               problem__fourier_type=2,
                               problem__regularisation__class='LASSO',
                                problem__regularisation__weights__le=1e-8,
                               solver__algo='vanilla',
                               solver__reg_factor=0.0125
                               )
online2vanilla = offline.union(online2vanilla)
fig = plot_metrics(online2vanilla,('cost_off','cost'),('nb_run','reg_factor'),log=True)
fig = plot_metrics(online2vanilla,('psnr',),('nb_run','reg_factor'),log=False)
fig = plot_metrics(online2vanilla,('ssim',),('nb_run','reg_factor'),log=False)





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Use a basic preconditionner

In [ ]:
ds2 = Experience.objects.filter(mode='and',
                                problem__fourier_type =2,
                                problem__regularisation__class='GroupLASSO',
                                problem__regularisation__weights=5e-4,
                                solver__nb_run__in=[1,2])
ds2 = ds2.filter(mode='loose_and',
                                solver__algo__in=['rmsprop','momentum'],
                                solver__gamma=1.00,
                                solver__eta=1.00,
                                solver__beta=1.00
                                )
ds2 = ds2.union(online2vanilla)
fig = plot_metrics(ds2,('psnr','ssim'),('nb_run','beta','gamma',),default_offset=80,log=False)
fig = plot_metrics(ds2,('cost_off',),('nb_run','beta','gamma',),default_offset=80,log=True)

## No regularisation

In [ ]:

offline_no_reg = Experience.objects.filter(mode='and',
                               problem__fourier_type__in=[0, 1, 2],
                               problem__regularisation__class='IdentityProx',
                               solver__algo='vanilla',
                               solver__reg_factor=1.,
                               )
ds2 = Experience.objects.filter(mode='loose_and',
                                problem__fourier_type=2,
                                problem__regularisation__class='IdentityProx',
                                solver__algo__in=['rmsprop', 'vanilla','momentum'],
                                solver__eta=1.0,
                                solver__gamma=1.00,
                                solver__reg_factor=1.,
                                solver__beta=1.00
                                )
ds2 = ds2.union(offline_no_reg)
fig = plot_metrics(ds2, ('psnr', 'ssim'), ('nb_run','reg_factor'),log=False)
fig2 = plot_metrics(ds2, ('cost_off','cost'), ('nb_run', 'reg_factor'),log=True)
fig2 = plot_metrics(ds2, ('data_res_on','data_res_off'), ('nb_run','reg_factor' ),log=True)

In [ ]:
# ADAM 

In [ ]:
ds2 = Experience.objects.filter(mode='and',
                                problem__fourier_type =2,
                                problem__regularisation__class='GroupLASSO',
                                problem__regularisation__weights=5e-4,)
ds2 = ds2.filter(mode='loose_and',
                                solver__algo__in=['adam',],
                                solver__reg_factor=0.0125,
                                solver__eta=1.00,
                                solver__beta=0.5,
                                solver__gamma=0.5
                                )
ds2 = ds2.union(offline)
fig = plot_metrics(ds2,('psnr','ssim'),('nb_run','fourier_type','reg_factor'),default_offset=0,log=False)

In [ ]:
from array2gif import write_gif
import numpy as np
from IPython.display import Image


def make_gif(exp, **kwargs):
    """
    Create a gif from the estimates
    """
    estimates = abs(np.array(exp.xf))
    estimates = np.repeat(estimates[:,np.newaxis,...],3,axis=1)
    estimates *= (255.0/estimates.max())
    print(estimates.shape)
    filename = f"{hash(exp)}.gif"
    write_gif(estimates, filename, **kwargs)
    

for exp in online2vanilla:
    make_gif(exp,fps=1)
    Image(f"{hash(exp)}.gif")